In [1]:
#coding=utf-8
import re, os, fnmatch, time, collections, pickle
import pandas as pd
import numpy as np
import commands
import collections, copy
from pandas import DataFrame, Series
import commands
use_delta = False
wkdir = "/store1/chenqy/linuxhistory/"
repodir = wkdir + 'linux/'
mtnrdir = wkdir + 'maintainer/'
revdir = mtnrdir + 'rev/'
lstreedir = mtnrdir + 'tree/'
idmp_fn = 'all.aliase.id.networkx.full'
hist_fn = 'MAINTAINERS.h.at.ct'
diff_fn = 'diff_maintainer.csv'
mtnr_stat_fn = 'mtnr_stat.csv'
delta_files_fn = 'unique-files.csv'
delta_files_rev_ct_fn = 'delta-files-rev-ct.csv'
delta_files_rev_ct_mtnrs_fn = 'delta-files-rev-ct-mtnrs.csv'
stat_my_fn = "mtnr-yearly-stat.csv"
stat_mym_fn = 'mtnr-monthly-stat.csv'
if not use_delta:
    delta_files_rev_ct_fn = 'lg-files-rev-ct.csv'
    delta_files_rev_ct_mtnrs_fn = 'lg-files-rev-ct-mtnrs.csv'
    stat_my_fn = "mtnr-yearly-stat-anyext.csv"
    stat_mym_fn = 'mtnr-monthly-stat-anyext.csv'

os.chdir(repodir)
log_cmd = "git log --no-merges --pretty=format:'%H;%at;%ct' MAINTAINERS > {}".format(hist_fn)
show_cmd = "git show {rev}:MAINTAINERS > {revdir}{rev}"
ls_tree_cmd = 'git ls-tree -r --name-only {rev} > {lstreedir}{rev}'
#commands.getoutput(log_cmd)


In [99]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/pkuas/chenqy/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['copy', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# with open(mtnrdir + hist_fn) as fin:
#     while True:
#         line = fin.readline()
#         if not line:
#             print 'Done'
#             break
#         rev = line.split(';')[0]
#         cmd = show_cmd.format(repodir=repodir, rev=rev, revdir=revdir)
#         commands.getoutput(cmd)


In [3]:
# with open(mtnrdir + hist_fn) as fin:
#     while True:
#         line = fin.readline()
#         if not line:
#             print 'Done'
#             break
#         rev = line.split(';')[0]
#         cmd = ls_tree_cmd.format(rev=rev, lstreedir=lstreedir)
#         commands.getstatusoutput(cmd)

In [2]:
tmp = pd.read_csv(wkdir + idmp_fn, sep=',', names=['frm', 'to'])
idmp = pd.Series(tmp.to.values, index=tmp.frm.values)

In [3]:
diff_col_names = ['sys_name', 'type', 'obj', 'op', 'rev', 'time', 'm_name', 'm_email'] 

In [4]:
def trans_to_id(name_email):
    if name_email[0] in idmp.index:
        return idmp.loc[name_email[0]]
    elif name_email[1] in idmp.index:
        return idmp.loc[name_email[1]]
    return name_email[1] if name_email[1] != '' else name_email[0]

class SubSys:
    def __init__(self, name):
        self.name = name
        self.mtnrs = dict()
        self.teams = dict()
        self.files = []
        self.files_excluded = []
        self.files_regex = []
        self.status = ''
        # self.mtnrs = mtnrs ?
        # self.files = files

def is_team(name_email):
    name, email = name_email
    return re.search('(corp|company|project|team|inc|corporation|incorporation)\.?$', name_email[0]) != None

def retrieve_mntr(line):
    line = line[2:].strip().lower()
    if '<' in line:
        tmp = re.split('<|>', re.sub(',', ' ', line))
        if len(tmp) >= 3:
            return tmp[0].strip(), tmp[1].strip()
    tmp = re.split(',', line)
    if len(tmp) >= 2:
        return tmp[0].strip(), tmp[1].strip()
    if '@' in line:
        return '', line
    return line, ''
    # m = re.match('(.*?),?\s+<(.*?)>', line)
    # if m == None:
    #     print 1
    #     m = re.match('([^@ ]+?),?\s+([^ ]*?)', line)
    # if m == None:
    #     print 2
    #     m = re.match('([^@ ]+?)\s*,\s*([^ ]*?)', line)
    # if m == None:
    #     print 3
    #     m = re.match('(\s*)([^ ]+)', line)
    # return ('', '') if m == None else m.groups()

def retrieve_file(line):
    return line[2:].strip()

def retrieve_status(line):
    return line[2:].strip()

def retrieve_subsys_mtnr_file(fn):
    res = dict()
    fin = open(fn)
    lines = fin.readlines()
    fin.close()
    nline, i = len(lines), 0
    nline_1 = nline - 1
    while i < nline and (not((len(lines[i].strip()) == 1 or (len(lines[i]) >= 2 and lines[i][1] != ':')) and i < nline_1 and len(lines[i + 1]) >= 2 and lines[i + 1][1] == ':')): # subsys not reached
        i += 1
    tmp = [l.strip() for l in lines]
    lines = tmp
    while i < nline:
        name = lines[i]
        #print 'SubSys:', name
        if name == '':
            continue
        tmpsys = SubSys(name)
        i += 1
        while i < nline and lines[i] != '':
            tl = len(lines[i])
            if tl >= 2:
                typeflag = lines[i][:2]
                if typeflag in ['P:', 'M:']:
                    name_email = retrieve_mntr(lines[i])
                    tmp_id = trans_to_id(name_email)
                    if is_team(name_email):
                        tmpsys.teams[tmp_id] = name_email
                    elif typeflag == 'P:' or (name_email[0] != ''):
                        #print 'tmp_id', lines[i], tmp_id, name_email
                        tmpsys.mtnrs[tmp_id] = name_email
                elif typeflag == 'F:':
                    tmpsys.files.append(retrieve_file(lines[i]))
                elif typeflag == 'X:':
                    tmpsys.files_excluded.append(retrieve_file(lines[i]))
                elif typeflag == 'N:':
                    tmpsys.files_regex.append(retrieve_file(lines[i]))
                elif typeflag == 'S:':
                    tmpsys.status = retrieve_status(lines[i])
            i += 1
        res[name] = tmpsys
        while i < nline and lines[i] == '':
            i += 1
    return res

# type: M, maintainer; F, file; S: sub system
col_names = ['sys_name', 'type', 'obj', 'op', 'rev', 'time', 'm_name', 'm_email'] # if type is not M, m_email and m_name are ''
def diff_rev(prev, now, prev_rev, now_rev, prev_time, now_time, op_type='+'):
    df_dict = dict()
    pos = 0
    for sys_name in now:
        if sys_name not in prev:
            df_dict[pos] = (sys_name, 'S', sys_name, op_type, now_rev, now_time, '', '')
            pos += 1
            dmtnrs = now[sys_name].mtnrs.keys()
            dfiles = now[sys_name].files
            dfiles_excluded = now[sys_name].files_excluded
            dfiles_regex = now[sys_name].files_regex
            dteams = now[sys_name].teams.keys()
            dstatus = [now[sys_name].status]
        else:
            dmtnrs = [k for k in now[sys_name].mtnrs if k not in prev[sys_name].mtnrs]
            dfiles = [f for f in now[sys_name].files if f not in prev[sys_name].files]
            dfiles_excluded = [f for f in now[sys_name].files_excluded if f not in prev[sys_name].files_excluded]
            dfiles_regex = [f for f in now[sys_name].files_regex if f not in prev[sys_name].files_regex]
            dteams = [t for t in now[sys_name].teams if t not in prev[sys_name].teams]
            dstatus = list() if now[sys_name].status == prev[sys_name] else [now[sys_name]]
        for m in dmtnrs:
            tm = now[sys_name].mtnrs[m]
            df_dict[pos] = (sys_name, 'M', m, op_type, now_rev, now_time, tm[0], tm[1])
            pos += 1
        for f in dfiles:
            df_dict[pos] = (sys_name, 'F', f, op_type, now_rev, now_time, '', '')
            pos += 1
        for f in dfiles_regex:
            df_dict[pos] = (sys_name, 'N', f, op_type, now_rev, now_time, '', '')
            pos += 1
        for f in dfiles_excluded:
            df_dict[pos] = (sys_name, 'X', f, op_type, now_rev, now_time, '', '')
            pos += 1
        for t in dteams:
            tt = now[sys_name].teams[t]
            df_dict[pos] = (sys_name, 'T', t, op_type, now_rev, now_time, tt[0], tt[1])
            pos += 1
        for st in dstatus:
            df_dict[pos] = (sys_name, 'ST', st, op_type, now_rev, now_time, '', '')
            pos += 1
            
    if len(df_dict.keys()) > 0:
        df = pd.DataFrame(df_dict.values(), columns=col_names)
    else:
        df = pd.DataFrame({}, columns=col_names)
    if op_type == '+': # find the infos deleted
        df = df.append(diff_rev(now, prev, prev_rev, now_rev, prev_time, now_time, op_type = '-')) # just switch now and prev. no need to switch rev and time
    return df


In [6]:
os.chdir(revdir)
rev_df = pd.read_csv(mtnrdir + hist_fn, sep=';', names=['rev', 'atm', 'ctm'])
#rev_df.sort_values('ctm', inplace=True)
rev_df.sort_values('ctm', inplace=True)
rev_df.index = rev_df.rev
revs = rev_df.rev
prev = dict()
prev_rev, prev_time = 0, 0
if os.path.exists(diff_fn):
    os.remove(diff_fn)

In [7]:
mtnr_rev = dict()
df_dict = dict()
revs_sorted = revs
for i, rev in enumerate(revs[:]): # 498:502
    #print 'processing', i, rev
    now_time, now_rev = rev_df.ctm.loc[rev], rev
    now = retrieve_subsys_mtnr_file(rev) # file name is rev
    mtnr_rev[rev] = now
    tdf = diff_rev(prev, now, prev_rev, now_rev, prev_time, now_time, op_type='+')
    df_dict[rev] = tdf
    prev, prev_time, prev_rev = now, now_time, now_rev

df = pd.concat(df_dict.values())#, columns=col_names)
df.sort_values(['sys_name', 'time'], inplace=True)

In [8]:
df.shape

(6719590, 8)

In [480]:
df.to_csv(mtnrdir + diff_fn, mode='w', header=False, index=False, sep=';')
#df.to_csv(mtnrdir + diff_fn, mode='a' if os.path.exists(diff_fn) else 'w', header=False, index=False, sep=';')


v = '''
数据在pae上，路径为 /store1/chenqy/linuxhistory/maintainer/diff_maintainer.csv （32M）
该文件的一行表示：对于某个Subsyste，该subsystem自身、Maintainer或相关文件在某个时间点发生的某个提交（commit）中被更改（增、删）。

各个字段为：
- subsystem名称
- 更改的对象类型：S，表示Subsystem; M, 表示Maintainer; F，表示File; T, 表示team, project, corp.....; X, files excluded; N, regex files
- 更改的对象名称：当更改的对象类型为S时，该字段的值为subsystem名称；当更改的对象类型为M时，表示Maintainer（已通过简单的方法处理了一个名字对应多个邮箱、一个邮箱对应多个名字的情况）；当更改的对象类型为F时，表示增加或删除的文件
- 更改的类型：+，表示增加；-，表示删除
- 发生更改的commit的hash值
- 发生更改的时间：从1970-1-1算起的秒数
- 备注1：当更改的对象类型为M时，该字段表示Maintainer的名字，这个名字是直接从文件中提取出来的，未经过处理；不为M时，该字段为空
- 备注2：当更改的对象类型为M时，该字段表示Maintainer的邮箱，这个邮箱是直接从文件中提取出来的，未经过处理；不为M时，该字段为空

文件的行按照 subsystem名称 和 发生更改的时间 排序。

注意点：
- 一个Maintainer的名字和邮箱可能在不同的commit中发生更改

'''

In [10]:
# for each element of mtnr_rev, map F: to sub sys name
fcate_each_rev_2_subsys = dict()
for rev in mtnr_rev.keys():
    mr = mtnr_rev[rev]
    tfcate2subsys = collections.defaultdict(list)
    for sysname in mr.keys():
        ss = mr[sysname]
        for f in ss.files:
            tfcate2subsys[f].append(sysname)
    fcate_each_rev_2_subsys[rev] = tfcate2subsys

In [11]:
for rev in fcate_each_rev_2_subsys:
    tmp = fcate_each_rev_2_subsys[rev]
    for f in tmp:
        tmp[f] = list(np.unique(tmp[f]))
    fcate_each_rev_2_subsys[rev] = tmp

In [12]:
all_mtnrs_of_rev = dict()
for rev in mtnr_rev.keys():
    mr = mtnr_rev[rev]
    tmp = set()
    for sysname in mr.keys():
        tmp |= set(mr[sysname].mtnrs)
    all_mtnrs_of_rev[rev] = tmp

## first read (unique) filenames generated by R

### basic idea of map order
- filename -> 
- file cate labeled 'F:' ->
- subsystem ->
- maintainer

In [13]:
mh = pd.read_csv(mtnrdir + diff_fn, names=diff_col_names, sep=';')

In [14]:
mh.fillna({'m_name':'', 'm_email':''}, inplace=True)
1

1

In [15]:
ufcate = mh['obj'][mh['type'] == 'F'].unique()

In [16]:
ufcate = Series(ufcate)

In [17]:
ufcate = ufcate[np.logical_not(np.in1d(ufcate, ['*', '*/']))]

In [18]:
# def convert_wildcard_fn(fn):
#     if fn[-1] == '/':
#         return fn + '.*'
#     elif fn[-1] == '*':
#         return fn[:-1] + '[!/][!/]*'
#     return fn

In [19]:
# a = map(convert_wildcard_fn, ufcate)

In [20]:
# # translation between two different regex
# a = ufcate.str.replace('\.', '\.')
# a = a.str.replace('\?', '.')
# # seems translation needs more work
# ufcate = a

In [206]:
def calc_sys2mod(sysname=None, flist=[]):
    mods = Series(map(lambda x : re.sub('/.*', '', x), flist))
    #print mods
    mods_cnt = mods.value_counts().sort_values(ascending = False)
    #print mods_cnt
    for m in mods_cnt.index:
        if m not in ['Documentation', 'include']:
            return m
    if sysname == None:
        return mods_cnt.index[0] if len(mods_cnt) > 0 else ''
    tmp = sysname.split()
    x = {'driver':'drivers', 'arch':'arch'}
    for m in x:
        if m in tmp:
            return x[m]
    if len(mods_cnt) > 0:
        return mods_cnt.index[0]
    return ''


In [22]:
diff_col_names

['sys_name', 'type', 'obj', 'op', 'rev', 'time', 'm_name', 'm_email']

In [19]:
sys2mod = collections.defaultdict(str)

In [20]:
for sysname in set(mh.sys_name):
    flist = list(set(mh['obj'][np.logical_and(mh['type'] == 'F', mh['sys_name'] == sysname)]))
    sys2mod[sysname] = calc_sys2mod(sysname, flist)

In [25]:
Series(sys2mod.values()).value_counts()

drivers          1191
                  226
arch              209
fs                 61
net                57
sound              36
kernel             31
mm                 15
security           12
scripts             9
tools               9
Documentation       6
include             5
block               3
lib                 3
crypto              2
Makefile            2
*                   1
driver              1
certs               1
virt                1
dtype: int64

In [21]:
def my_fnmatch(fn, pattern):
    lenp = len(pattern)
    if lenp >= 1 and pattern[-1] == '/':
        return fnmatch.fnmatch(fn, pattern + '*')
    return fn.count('/') == pattern.count('/') and fnmatch.fnmatch(fn, pattern)    

In [22]:
re.sub("/.*", "", 'p/asdasd/x.c')

'p'

In [23]:
# get unique files from delta
# ufs = pd.read_csv(mtnrdir + delta_files_fn, names=['fn'], sep=',')
# ufs = ufs.fn

In [24]:
# get unifue files from ls-tree of all revs 
tree_of_rev = collections.defaultdict(list)
for r in mtnr_rev.keys():
    with open(lstreedir + r) as fin:
        tree_of_rev[r] = map(lambda x : x.strip(), fin.readlines())

In [25]:
ufs = reduce(lambda x, y : set(x) | set(y), tree_of_rev.values(), [])

In [26]:
ufs = Series(list(ufs))

In [472]:
np.in1d('drivers/infiniband/hw/usnic/usnic_debugfs.c', ufs)

array([ True], dtype=bool)

In [32]:
ufs.head()

0                              drivers/mfd/rt5033.c
1    drivers/net/ethernet/qlogic/qlcnic/qlcnic_hw.h
2              arch/mips/mti-sead3/sead3-platform.c
3                      include/asm-sparc64/hw_irq.h
4    drivers/net/ethernet/qlogic/qlcnic/qlcnic_hw.c
dtype: object

In [473]:
my_fnmatch('drivers/infiniband/hw/usnic/usnic_debugfs.c', 'drivers/infiniband/hw/usnic')

False

In [33]:
fcate_of_fn = collections.defaultdict(set)

In [34]:
for cate in ufcate:
    m = map(lambda x:my_fnmatch(x, cate), ufs)
    muf = ufs[m]
    for f in muf:
        fcate_of_fn[f].add(cate)

In [451]:
with open(mtnrdir + 'fcate_of_fn.pkl', 'w') as fout:
    pickle.dump(fcate_of_fn, fout)

In [27]:
with open(mtnrdir + 'fcate_of_fn.pkl', 'r') as fin:
    fcate_of_fn = pickle.load(fin)

In [28]:
fcate_of_fn.keys()[:5]

['drivers/mfd/rt5033.c',
 'drivers/net/ethernet/qlogic/qlcnic/qlcnic_hw.h',
 'arch/cris/kernel/process.c',
 'drivers/atm/idt77252.c',
 'drivers/net/ethernet/qlogic/qlcnic/qlcnic_hw.c']

In [36]:
fcate_of_fn['fs/inode.c']

{'fs/*'}

In [37]:
len(ufs), len(fcate_of_fn.keys())

(89203, 76452)

In [98]:
ks = fcate_of_fn.keys()
c = Series(map(lambda k:len(fcate_of_fn[k]), ks), index=ks)

In [99]:
c.describe()

count    89203.000000
mean         1.387274
std          0.908539
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          6.000000
dtype: float64

In [40]:
c[c>5]

arch/arm/kernel/perf_event.c    6
dtype: int64

In [41]:
fcate_of_fn['arch/arm/kernel/perf_event.c']

{'arch/*/kernel/perf_event*.c',
 'arch/*/kernel/perf_event.c',
 'arch/arm*/kernel/perf_*',
 'arch/arm/',
 'arch/arm/kernel/perf_*',
 'arch/arm/kernel/perf_event*'}

In [42]:
x = ufs[np.logical_not(np.in1d(ufs, fcate_of_fn.keys()))]

In [43]:
len(x)

12751

In [560]:
xtmp = delta_file_rev_ct.copy()

In [29]:
delta_file_rev_ct = pd.read_csv(mtnrdir + delta_files_rev_ct_fn, names=['fn', 'rev', 'cid','ctm', 'aid', 'atm'], sep=',')

/home/pkuas/chenqy/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
delta_file_rev_ct.shape

(1305334, 6)

In [31]:
delta_file_rev_ct.sort_values(['ctm', 'rev'], inplace=True)
1

1

In [47]:
# for c in xtmp.columns:
#     if c not in delta_file_rev_ct.columns:
#         delta_file_rev_ct[c] = xtmp[c]

In [32]:
delta_file_rev_ct['mtnrs'] = ''

In [49]:
delta_file_rev_ct.head()

,fn,rev,cid,ctm,aid,atm,mtnrs
1288044,fs/cifs/asn1.c,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,
1288045,include/asm-m32r/resource.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,
1288046,arch/ppc/platforms/pal4_pci.c,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,
1288047,include/net/sctp/command.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,
1288048,drivers/isdn/hisax/ipac.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,


#### steps:
- use ctm to locate an mtnr_rev: the newest mtnr_rev that is older than ctm 
- fn -> fcate -> subsys -> maintainer list

In [33]:
def get_maintainer_list_and_subsys_list(fn, fcate_of_fn, fcate_2_subsys, mntr_of_rev):
    if fn not in fcate_of_fn:
        return idmp['linus torvalds'], Series()
    fcates = fcate_of_fn[fn]
    ret = []
    subsys_list = []
    for fc in fcates:
        if fc in fcate_2_subsys:
            subsys = fcate_2_subsys[fc]
            subsys_list.extend(subsys)
            for ss in subsys:
                ret.extend(mntr_of_rev[ss].mtnrs)

    return (idmp['linus torvalds'] if len(ret) == 0 else '#'.join(np.unique(ret))), Series(subsys_list)

In [34]:
tmtnrs_list = [''] *  len(delta_file_rev_ct.index)

In [35]:
trevs_list = [''] * len(delta_file_rev_ct.index)

In [36]:
status_list = [''] * len(delta_file_rev_ct.index)

In [37]:
subsys_list = [''] * len(delta_file_rev_ct.index)

In [38]:
stidx = delta_file_rev_ct.shape[0] - 3

In [39]:
idx = 0

In [40]:
itr = delta_file_rev_ct.iloc[idx:].iterrows()

In [41]:
itr_rev_df = rev_df.iterrows()

In [42]:
one_rev_of_mtnr = next(itr_rev_df)[1]
one_rev_of_mtnr1 = next(itr_rev_df)[1]
tmtnr = one_rev_of_mtnr.ctm
tmtnr1 = one_rev_of_mtnr1.ctm

In [43]:
tdelta = delta_file_rev_ct.ctm.iloc[idx]

In [44]:
while tdelta  < tmtnr:
    one_rev_of_mtnr = one_rev_of_mtnr1
    one_rev_of_mtnr1 = next(itr_rev_df)[1]
    tmtnr = one_rev_of_mtnr.ctm
    tmtnr1 = one_rev_of_mtnr1.ctm

In [155]:
rev_df[rev_df.ctm >= 1267221550].head()

,rev,atm,ctm
rev,,,
a040d532b912b5dd7d88692b580cff9c88b987e3,a040d532b912b5dd7d88692b580cff9c88b987e3,1266914078,1267221550
ddf0289db22c4ae2192a252706792837528605c6,ddf0289db22c4ae2192a252706792837528605c6,1261344247,1267457742
661cb9fbf2ce580000a792bf1d394fc341c9ba69,661cb9fbf2ce580000a792bf1d394fc341c9ba69,1267470228,1267470276
e8e6cb325671daa17c776ad53b70fd6213260368,e8e6cb325671daa17c776ad53b70fd6213260368,1262676506,1267486855
4c32531324b83672f100692354b680625bcd7fba,4c32531324b83672f100692354b680625bcd7fba,1267692150,1267692834


In [157]:
time.gmtime(1261344247), time.gmtime(1267457742)

(time.struct_time(tm_year=2009, tm_mon=12, tm_mday=20, tm_hour=21, tm_min=24, tm_sec=7, tm_wday=6, tm_yday=354, tm_isdst=0),
 time.struct_time(tm_year=2010, tm_mon=3, tm_mday=1, tm_hour=15, tm_min=35, tm_sec=42, tm_wday=0, tm_yday=60, tm_isdst=0))

In [45]:
tdelta, tmtnr, tmtnr1

(1113690036, 1113690036, 1113690356)

In [46]:
while True:
    try:
        stop_flag = 1
        #print '1'
        onedelta = next(itr)[1]
        #print onedelta
        stop_flag = 2
        #print '2'
        while onedelta.ctm >= tmtnr1:
            one_rev_of_mtnr = one_rev_of_mtnr1
            one_rev_of_mtnr1 = next(itr_rev_df)[1]
            tmtnr1 = one_rev_of_mtnr1.ctm
        #print '3'
        tmtnr = one_rev_of_mtnr.ctm
        #print one_rev_of_mtnr.rev
        #def get_maintainer_list_and_subsys_list(fn, fcate_of_fn, fcate_2_subsys, mntr_of_rev):
        tmtnrs_list[idx], subsys_list[idx] = get_maintainer_list_and_subsys_list(onedelta.fn, fcate_of_fn, fcate_each_rev_2_subsys[one_rev_of_mtnr.rev], mtnr_rev[one_rev_of_mtnr.rev])
        trevs_list[idx] = one_rev_of_mtnr.rev
        #status_list[idx] = 'Unknown' if len(subsys_list[idx]) == 0 else mtnr_rev[one_rev_of_mtnr.rev][subsys_list[idx].value_counts().sort_values(ascending=False).index[0]].status
        status_list[idx] = 'Unknown' if len(subsys_list[idx]) == 0 else '#'.join([mtnr_rev[one_rev_of_mtnr.rev][sysname].status for sysname in subsys_list[idx]])
        idx += 1
        if idx % 10000 == 0:
            print idx, 
    except StopIteration:
        if stop_flag == 2:
            tmtnr = one_rev_of_mtnr.ctm
            #def get_maintainer_list_and_subsys_list(fn, fcate_of_fn, fcate_2_subsys, mntr_of_rev):
            while True:
                try:
                    tmtnrs_list[idx], subsys_list[idx] = get_maintainer_list_and_subsys_list(onedelta.fn, fcate_of_fn, fcate_each_rev_2_subsys[one_rev_of_mtnr.rev], mtnr_rev[one_rev_of_mtnr.rev])
                    trevs_list[idx] = one_rev_of_mtnr.rev
                    status_list[idx] = 'Unknown' if len(subsys_list[idx]) == 0 else '#'.join([mtnr_rev[one_rev_of_mtnr.rev][sysname].status for sysname in subsys_list[idx]])
                    idx += 1
                    onedelta = next(itr)[1]
                    #print onedelta
                except StopIteration:
                    break
        break

10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 480000 490000 500000 510000 520000 530000 540000 550000 560000 570000 580000 590000 600000 610000 620000 630000 640000 650000 660000 670000 680000 690000 700000 710000 720000 730000 740000 750000 760000 770000 780000 790000 800000 810000 820000 830000 840000 850000 860000 870000 880000 890000 900000 910000 920000 930000 940000 950000 960000 970000 980000 990000 1000000 1010000 1020000 1030000 1040000 1050000 1060000 1070000 1080000 1090000 1100000 1110000 1120000 1130000 1140000 1150000 1160000 1170000 1180000 1190000 1200000 1210000 1220000 1230000 1240000 1250000 1260000 1270000 1280000 1290000 1300000


In [47]:
subsys_list_of_idx = dict()
for i in range(len(subsys_list)):
    subsys_list_of_idx[delta_file_rev_ct.index[i]] = subsys_list[i]

In [65]:
reduce(lambda x, y: x.append(y), [Series([1, 2]), Series([3])])

0    1
1    2
0    3
dtype: int64

In [66]:
tmtnrs_list[-5:]

['seth jennings',
 'dmitry torokhov',
 'w.sang@pengutronix.de',
 'a.zummo@towertech.it#alexandre belloni',
 'a.zummo@towertech.it#alexandre belloni#support.opensource@diasemi.com']

In [67]:
trevs_list[-5:]

['13cbd906e89a1af42c586dcc4f42b311d23b7fd1',
 '13cbd906e89a1af42c586dcc4f42b311d23b7fd1',
 '13cbd906e89a1af42c586dcc4f42b311d23b7fd1',
 '13cbd906e89a1af42c586dcc4f42b311d23b7fd1',
 '13cbd906e89a1af42c586dcc4f42b311d23b7fd1']

In [48]:
tmtnrs_list_back = copy.copy(tmtnrs_list)

In [49]:
tmtnrs_list_tmp = [idmp['linus torvalds'] if i == '' else i for i in tmtnrs_list]

In [50]:
delta_file_rev_ct['mtnrs'] = tmtnrs_list_tmp

In [51]:
delta_file_rev_ct['mrev'] = trevs_list

In [52]:
delta_file_rev_ct['num_mtnr'] = map(lambda x:x.count('#') + 1, delta_file_rev_ct['mtnrs'])

In [73]:
idmp[idmp=='alexandre belloni']

alexandre belloni                       alexandre belloni
alexandre.belloni@free-electrons.com    alexandre belloni
dtype: object

- at the file level:  how much deltas each file has, what fraction is 
done by ownMaintainers/otherMaintainers/pureAuthors, how many 
ownMaintainers/otherMaintainers/pureAuthors contributed (authored), how 
many committers, how often its maintainers change?

- at the maintainer level: how many files a maintainer has, how much 
code is there, how many deltas he authored, how many authors he 
committed code for, the first time he was added to the maintainers file 
and the last time he was removed from the file


In [53]:
imlist = dict()
for i in delta_file_rev_ct.index:
    imlist[i] = delta_file_rev_ct['mtnrs'].loc[i].split('#')

In [54]:
all_mlist = list({m for i in imlist for m in imlist[i]})

In [55]:
len(all_mlist)

1315

In [56]:
def check_author_is_own_mtnr(i):
    return delta_file_rev_ct.aid[i] in imlist[i]

def check_author_is_any_mtnr(i):
    return delta_file_rev_ct.aid[i] in all_mtnrs_of_rev[delta_file_rev_ct.mrev[i]]

In [57]:
delta_file_rev_ct['ownMtnr'] = map(check_author_is_own_mtnr, delta_file_rev_ct.index)

In [58]:
delta_file_rev_ct['anyMtnr'] = map(check_author_is_any_mtnr, delta_file_rev_ct.index)

In [59]:
delta_file_rev_ct['ownMtnr'] += 0

In [60]:
delta_file_rev_ct['anyMtnr'] += 0

In [61]:
delta_file_rev_ct['status'] = status_list

In [1264]:
# at file level


In [1265]:
delta_file_rev_ct.to_csv(mtnrdir + delta_files_rev_ct_mtnrs_fn, mode='w', header=False, index=False, sep=';', quoting=None)

In [83]:
14.97/49.9, 11.66/36

(0.30000000000000004, 0.3238888888888889)

In [84]:
delta_file_rev_ct.shape

(1305334, 12)

In [85]:
fgrp = delta_file_rev_ct.head(10).groupby('fn')

In [86]:
delta_file_rev_ct.tail(3)

,fn,rev,cid,ctm,aid,atm,mtnrs,mrev,num_mtnr,ownMtnr,anyMtnr,status
2,drivers/i2c/busses/i2c-rcar.c,b4cd08aa1f53c831e67dc5c6bc9f9acff27abcba,w.sang@pengutronix.de,1450522837,w.sang@pengutronix.de,1450292718,w.sang@pengutronix.de,13cbd906e89a1af42c586dcc4f42b311d23b7fd1,1,1,1,Maintained#Maintained
1,drivers/rtc/rtc-rk808.c,f076ef44a44d02ed91543f820c14c2c7dff53716,alexandre belloni,1450615140,julius werner,1450220569,a.zummo@towertech.it#alexandre belloni,13cbd906e89a1af42c586dcc4f42b311d23b7fd1,2,0,0,Maintained
0,drivers/rtc/rtc-da9063.c,77535acedc26627f16a1a39c1471f942689fe11e,alexandre belloni,1450615169,opensource [steve twiss],1449592119,a.zummo@towertech.it#alexandre belloni#support...,13cbd906e89a1af42c586dcc4f42b311d23b7fd1,3,0,0,Maintained#Supported


In [87]:
fgrp.fn.agg(tf)

NameError: name 'tf' is not defined

## maintainer level

In [592]:
os.chdir(repodir)
ftmp = commands.getoutput("find . -type f -not -path '*/\.*'")

In [593]:
all_files = ftmp.split('\n')

In [596]:
all_files[:5]

['./arch/alpha/boot/bootloader.lds',
 './arch/alpha/boot/bootp.c',
 './arch/alpha/boot/bootpz.c',
 './arch/alpha/boot/head.S',
 './arch/alpha/boot/main.c']

In [597]:
all_files = map(lambda x:x[2:], all_files)

In [598]:
len(all_files)

52017

In [599]:
all_files[:5]

['arch/alpha/boot/bootloader.lds',
 'arch/alpha/boot/bootp.c',
 'arch/alpha/boot/bootpz.c',
 'arch/alpha/boot/head.S',
 'arch/alpha/boot/main.c']

In [846]:
delta_file_rev_ct.tail()

,fn,rev,cid,ctm,aid,atm,mtnrs,mrev,num_mtnr,ownMtnr,anyMtnr,status,year
4,mm/zswap.c,8bc8b228d076ae93398316f81eab35f3d12c0c4f,linus torvalds,1450477540,dan streetman,1450477324,seth jennings,13cbd906e89a1af42c586dcc4f42b311d23b7fd1,1,0,1,Maintained,2015
3,drivers/input/touchscreen/elants_i2c.c,478e5ed1c3f6928ece7fffd712ba728b1f92217d,dmitry torokhov,1450484503,james chen,1450482708,dmitry torokhov,13cbd906e89a1af42c586dcc4f42b311d23b7fd1,1,0,0,Maintained,2015
2,drivers/i2c/busses/i2c-rcar.c,b4cd08aa1f53c831e67dc5c6bc9f9acff27abcba,w.sang@pengutronix.de,1450522837,w.sang@pengutronix.de,1450292718,w.sang@pengutronix.de,13cbd906e89a1af42c586dcc4f42b311d23b7fd1,1,1,1,Maintained,2015
1,drivers/rtc/rtc-rk808.c,f076ef44a44d02ed91543f820c14c2c7dff53716,alexandre belloni,1450615140,julius werner,1450220569,a.zummo@towertech.it#alexandre belloni,13cbd906e89a1af42c586dcc4f42b311d23b7fd1,2,0,0,Maintained,2015
0,drivers/rtc/rtc-da9063.c,77535acedc26627f16a1a39c1471f942689fe11e,alexandre belloni,1450615169,opensource [steve twiss],1449592119,a.zummo@towertech.it#alexandre belloni#support...,13cbd906e89a1af42c586dcc4f42b311d23b7fd1,3,0,0,Supported,2015


## yearly maintainer stats

In [62]:
delta = copy.copy(delta_file_rev_ct)

In [63]:
delta['year'] = map(lambda t : time.gmtime(t).tm_year, delta['ctm'])

In [481]:
time.gmtime(1113690036)

time.struct_time(tm_year=2005, tm_mon=4, tm_mday=16, tm_hour=22, tm_min=20, tm_sec=36, tm_wday=5, tm_yday=106, tm_isdst=0)

In [64]:
#delta['ym'] = delta['year'].astype(str) + '.' + map(lambda t : '{:0>2}'.format(time.gmtime(t).tm_mon), delta['ctm'])
delta['ym'] = delta['year'] + map(lambda t : time.gmtime(t).tm_mon / 100.0, delta['ctm'])

In [80]:
mh['year'] = map(lambda t : time.gmtime(t).tm_year, mh['time'])
mh['ym'] = mh['year'] + map(lambda t : time.gmtime(t).tm_mon / 100.0, mh['time'])

In [92]:
delta.head()

,fn,rev,cid,ctm,aid,atm,mtnrs,mrev,num_mtnr,ownMtnr,anyMtnr,status,year,ym
1288044,fs/cifs/asn1.c,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04
1288045,include/asm-m32r/resource.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04
1288046,arch/ppc/platforms/pal4_pci.c,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04
1288047,include/net/sctp/command.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04
1288048,drivers/isdn/hisax/ipac.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04


In [ ]:
# imlist = dict()
# for i in delta.index:
#     imlist[i] = delta['mtnrs'].loc[i].split('#')

In [275]:
Series(map(lambda i:len(subsys_list_of_idx[i]), subsys_list_of_idx.keys())).describe()

count    1.305334e+06
mean     1.010661e+00
std      9.374413e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      5.000000e+00
dtype: float64

In [66]:
idx_of_mtnr_year = collections.defaultdict(list)

In [67]:
for idx in delta.index:
    iml = delta['mtnrs'].loc[idx].split('#')
    for m in iml:
        idx_of_mtnr_year[(m, delta['year'].loc[idx])].append(idx)

In [284]:
('3chas3@gmail.com', 2005) in idx_of_mtnr_year, ('3chas3@gmail.com', 2005) in  stat_of_mtnr_year

(False, False)

In [292]:
stat_of_mtnr_year = collections.defaultdict(dict)

In [293]:
for my in idx_of_mtnr_year.keys():
#     v = reduce(lambda x, y : list(set(x + y)), [list(subsys_list_of_idx[i]) for i in idx_of_mtnr_year[my]], [])
#     stat_of_mtnr_year[my]['subsys_list'] = v
    t = list(set(delta['fn'].loc[idx_of_mtnr_year[my]]))
    stat_of_mtnr_year[my]['file_list'] = t    
    z = list(set(delta['fn'].loc[idx_of_mtnr_year[my]][delta['status'].loc[idx_of_mtnr_year[my]].str.contains('Supported')]))
    stat_of_mtnr_year[my]['supported_file_list'] = z

In [294]:
#stat_my.subsys_list[:4].str

In [295]:
def my_add(x, k, v, f, container):
    if k not in x:
        x[k] = container()
    eval('x[k].' + f + '(' + v + ')')

In [ ]:
'''
按照修改文件来计算时，只要一个maintainer出现在MAINTAINERS文件中，就会为这个maintainer加1条当年的记录
'''

In [296]:
for my in stat_of_mtnr_year.keys():
    stat_of_mtnr_year[my]['subsys_list'] = set()
    stat_of_mtnr_year[my]['subsys_file_list'] = collections.defaultdict(set)
    stat_of_mtnr_year[my]['subsys_supported_file_list'] = collections.defaultdict(set)

for rev in mtnr_rev.keys():
    try:
        y = mh['year'][mh['rev'] == rev].iloc[0] # mh may not have record of rev
    except:
        continue
    mrev = mtnr_rev[rev]
    for sysname in mrev:
        subsys = mrev[sysname]
        for m in subsys.mtnrs.keys():
            my = (m, y)
            if my not in stat_of_mtnr_year:
                stat_of_mtnr_year[my] = {'subsys_list':set(), 'file_list':set(), 'supported_file_list':set(), 
                                         'subsys_file_list':collections.defaultdict(set), 
                                        'subsys_supported_file_list':collections.defaultdict(set)}
            
            stat_of_mtnr_year[my]['subsys_list'].add(sysname)
            for f in stat_of_mtnr_year[my]['file_list']:
                if f not in fcate_of_fn:
                    stat_of_mtnr_year[my]['subsys_file_list']['THE REST'].add(f)
                else:
                    fca = fcate_of_fn[f]
                    for ifca in fca:
                        if ifca in subsys.files:
                            stat_of_mtnr_year[my]['subsys_file_list'][sysname].add(f)
                            if subsys.status == 'Supported':
                                stat_of_mtnr_year[my]['subsys_supported_file_list'][sysname].add(f)


In [288]:
len(fcate_of_fn)

76452

In [290]:
len(np.unique(delta.fn))

91550

In [ ]:
for i in range(0, 100, 10):
    v = stat_of_mtnr_year[stat_of_mtnr_year.keys()[i]]
    print 'file_list', len(v['file_list'])
    print 'supported_file_list', len(v['supported_file_list'])
    print 'subsys_file_list', sum([len(v['subsys_file_list'][k]) for k in v['subsys_file_list']])
    print 'subsys_supported_file_list', sum([len(v['subsys_supported_file_list'][k]) for k in v['subsys_supported_file_list']])
    print 

In [297]:
for my in stat_of_mtnr_year.keys():
    try:
        m_prevy = (my[0], my[1] - 1)
        if m_prevy not in idx_of_mtnr_year:
            stat_of_mtnr_year[my]['added_file_list'] = stat_of_mtnr_year[my]['file_list']
            stat_of_mtnr_year[my]['deled_file_list'] = []
        else:
            stat_of_mtnr_year[my]['added_file_list'] = list(np.setdiff1d(stat_of_mtnr_year[my]['file_list'], stat_of_mtnr_year[m_prevy]['file_list']))
            stat_of_mtnr_year[my]['deled_file_list'] = list(np.setdiff1d(stat_of_mtnr_year[m_prevy]['file_list'], stat_of_mtnr_year[my]['file_list']))
    except Exception, e:
            continue

In [298]:
res = []

In [299]:
for my in stat_of_mtnr_year:
    tmp = dict()
    try:
        tmp['mtnr'], tmp['year'] = my[0], my[1]
    except Exception:
        continue
    tmp['num_subsys'] = len(stat_of_mtnr_year[my]['subsys_list'])
    tmp['subsys_list'] = '#'.join(stat_of_mtnr_year[my]['subsys_list'])
    subsys_f_cnt = 0
    for (modi, k) in [('mod1', 'subsys_file_list'), ('mod2', 'subsys_supported_file_list')]:
        t = stat_of_mtnr_year[my][k]
        tmp[k + '_cnt'] = len(reduce(lambda x, y : set(list(x) + list(y)), t.values(), []))
        if len(t) <= 0:
            continue
        x = Series(map(lambda v:len(t[v]), t.keys()), index=t.keys())
        tmp[modi] = sys2mod[x.idxmax()]
        
    
    for k in ['file_list', 'supported_file_list', 'added_file_list', 'deled_file_list']:
        tmp['num_' + k] = len(stat_of_mtnr_year[my][k])
    
    res.append(tmp)
    

In [300]:
stat_my = DataFrame(res)

In [301]:
stat_my.sort_values(['mtnr', 'year'], inplace=True)
1

1

In [ ]:
stat_my[(stat_my['num_file_list'] != stat_my['subsys_file_list_cnt'])]
1

In [ ]:
stat_my.head(10)[new_cols]

In [302]:
new_cols = ['mtnr', 'year', 'num_subsys', 'subsys_list', 'mod1', 'mod2', 'num_file_list', 'num_supported_file_list', 
            'subsys_file_list_cnt', 'subsys_supported_file_list_cnt', 'num_added_file_list', 'num_deled_file_list']

In [303]:
stat_my.isnull().sum()

mod1                              2489
mod2                              5868
mtnr                                 0
num_added_file_list                  0
num_deled_file_list                  0
num_file_list                        0
num_subsys                           0
num_supported_file_list              0
subsys_file_list_cnt                 0
subsys_list                          0
subsys_supported_file_list_cnt       0
year                                 0
dtype: int64

In [304]:
stat_my.shape

(7833, 12)

In [305]:
stat_my.fillna('', inplace=True)
1

1

In [1296]:
stat_my[new_cols].to_csv(mtnrdir + stat_my_fn, sep=';', header=True, index=False)

### monthly stat

In [482]:
delta.shape

(1305334, 14)

In [327]:
idx_of_mtnr_month = collections.defaultdict(list)

In [328]:
for idx in delta.index:
    ym = delta.ym.loc[idx]
    for m in imlist[idx]:
        idx_of_mtnr_month[(m, ym)].append(idx)

In [329]:
# stat_of_mtnr_month = collections.defaultdict(dict)

In [330]:
# for mym in idx_of_mtnr_month:
#     m, ym = mym
#     for idx in idx_of_mtnr_month[mym]:
#         if mym not in stat_of_mtnr_month:
#             stat_of_mtnr_month[mym] = {'num_deltas_athr_mtnr':0, 'commits_athr_mtnr':set()}
#         stat_of_mtnr_month[mym]

In [ ]:
mdelta = delta[np.in1d(delta.aid, all_mlist)]

In [ ]:
mdelta.shape, delta.shape

In [ ]:
mdelta.head()

In [ ]:
grp = mdelta.groupby(['aid', 'ym', 'ownMtnr'])

In [ ]:
# def calc_stat_of_mtnr_month(x):
#     print x
#     return len(x.rev), len(set(x.rev)), sum(x.ownMtnr), len(set(x.rev[x.ownMtnr == 1]))

In [ ]:
v = grp.agg({'rev': lambda x : (len(x), len(set(x)))})
#v = grp.agg(calc_stat_of_mtnr_month)

In [ ]:
type(v), type(v.iloc[0])

In [ ]:
v.head()

In [ ]:
grp.groups.keys()[:5]

In [ ]:
v.rev.loc['broonie+linaro@kernel.org', 2015.1, 1]

In [ ]:
stat_of_mym = collections.defaultdict(dict)
for k in grp.groups.keys():
    m, ym, om = k
    mym = (m, ym)
    tmpv = v.rev.loc[m, ym, om]
    if mym not in stat_of_mym:
        stat_of_mym[mym] = {'num_deltas_athr_mtnr':0, 'num_commits_athr_mtnr':0, 'num_deltas_athr':0, 'num_commits_athr':0}
    if om == 1:
        stat_of_mym[mym]['num_deltas_athr_mtnr'] += tmpv[0]
        stat_of_mym[mym]['num_commits_athr_mtnr'] += tmpv[1]
    stat_of_mym[mym]['num_deltas_athr'] += tmpv[0]
    stat_of_mym[mym]['num_commits_athr'] += tmpv[1]
        

In [ ]:
res = []
for mym in stat_of_mym:
    x = stat_of_mym[mym]
    tmp_res = {'mtnr':mym[0], 'month':mym[1]}
    for k in x:
        tmp_res[k] = x[k]
    res.append(tmp_res)

In [ ]:
df_of_mym = DataFrame(res)[['mtnr', 'month', 'num_commits_athr', 'num_commits_athr_mtnr', 'num_deltas_athr', 'num_deltas_athr_mtnr']]

In [ ]:
df_of_mym.sort_values(['mtnr', 'month'], inplace=True)
1

In [ ]:
df_of_mym.head()

In [1312]:
df_of_mym.to_csv(mtnrdir + stat_mym_fn, mode='w', header=True, index=False, sep=';', quoting=None, float_format ='%.2f')

In [1313]:
#delta[delta.fn == 'drivers/watchdog/sc1200wdt.c']

## calc num of maintained files, by maintained, not only those modified

In [ ]:
fcate_of_fn_old = fcate_of_fn

In [230]:
fcate_of_fn = fcate_of_fn_old
x = {k:fcate_of_fn[k] for k in fcate_of_fn if len(fcate_of_fn[k]) > 0}
fcate_of_fn = x

In [68]:
res = []

In [69]:
len(tree_of_rev)

3436

In [73]:
for i in range(len(tree_of_rev.keys())):
    if i < newi:
        continue
    r = tree_of_rev.keys()[i]
    if i % 100 == 0:
        print (i, r), '*', 
    tr = tree_of_rev[r]
    fcate_2_subsys = fcate_each_rev_2_subsys[r]
    mntr_of_rev = mtnr_rev[r]
    for f in tr:
        if f not in fcate_of_fn:
            res.append({'rev':r, 'file':f, 'mtnr':'linus torvalds','subsys':'THE REST', 'status':'Unknown'})
        else:
            fcates = fcate_of_fn[f]
            for fc in fcates:
                if fc in fcate_2_subsys:
                    subsys = fcate_2_subsys[fc]
                    for ss in subsys:
                        status = mntr_of_rev[ss].status
                        for ms in mntr_of_rev[ss].mtnrs:
                            res.append({'rev':r, 'file':f, 'mtnr':ms, 'subsys':ss, 'status':status})
#         ml, sl = get_maintainer_list_and_subsys_list(f, fcate_of_fn, fcate_each_rev_2_subsys[r], mtnr_rev[r])
#         for m in ml:
#             res.append({'rev':r, 'mtnr':m, 'file':f})


(0, '81df63a9a5f98edbfaae70729147a33a3b7690a4') * (100, 'f1f6630b53e702648a248fecd881f3df5968a31a') * (200, 'a1cfac48ba4c7481bb749e0f4f37c85cb871b2d1') * (300, '22e7a424854b80f00bd5686b6539726b8ca95420') * (400, 'df4e33ad249b7480d6ba5dd11fc1dac5cfaffce1') * (500, '8fc1b0f87d9fcc7f05873c70b3003328c3d7defa') * (600, '04ac2f46d6ecb995f78c9ae4e2e4707d00b5339f') * (700, 'f7b98477f613a69b74ba4a715856630cc6508c0d') * (800, '8862bf1ed60de49550109b7023a0a33eb7db8b3c') * (900, '8eae0fb7efa99d55afa9e94ce95d101af0830ca6') * (1000, 'a14c0f8fefcb95847fe3a94a4085a7ceb88f19e5') * (1100, 'dc7dfcd838985947ee1a66cdaa41c4968404e0ed') * (1200, 'fbfd8b5622a18afefb53a69d31fbddecba404f64') * (1300, '1392707977de9e0df18123d56c3148cb0b1c6ab5') * (1400, 'db7bee24d23d82cc55c7cbc9a1f82d07066d6fce') * (1500, '48de5beeb0ee821859c79ba5b9462654604d8bf3') * (1600, '8a62ab4c4eaf5bce4d9cc84b77d6402c4742d9ab') * (1700, 'cec7c893d8654723028f09d33341e42673558057') * (1800, '9bb3c4469e317919b0fde8c0e0a3ebe7bd2cf167') * (1900

In [71]:
i = 0

In [72]:
newi = i

In [336]:
len(res)

217284141

In [74]:
mf = DataFrame(res)

In [75]:
mf.shape

(217284141, 5)

In [339]:
#mf.to_csv(mtnrdir + 'mf.txt', mode='w', header=True, index=False, sep=';')

In [142]:
mf.drop_duplicates(inplace=True)
1

1

In [483]:
mf.head()

,file,mtnr,rev,status,subsys
0,.gitignore,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST
1,.mailmap,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST
2,COPYING,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST
3,CREDITS,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST
4,Documentation/.gitignore,rlandley@parallels.com,81df63a9a5f98edbfaae70729147a33a3b7690a4,Maintained,DOCUMENTATION


In [341]:
delta.columns

Index([u'fn', u'rev', u'cid', u'ctm', u'aid', u'atm', u'mtnrs', u'mrev',
       u'num_mtnr', u'ownMtnr', u'anyMtnr', u'status', u'year', u'ym'],
      dtype='object')

In [384]:
t = delta[delta.fn == 'MAINTAINERS'][['rev', 'ctm', 'year']].drop_duplicates()

In [385]:
t.shape

(3452, 3)

In [484]:
delta.head()

,fn,rev,cid,ctm,aid,atm,mtnrs,mrev,num_mtnr,ownMtnr,anyMtnr,status,year,ym
1288044,fs/cifs/asn1.c,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04
1288045,include/asm-m32r/resource.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04
1288046,arch/ppc/platforms/pal4_pci.c,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04
1288047,include/net/sctp/command.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04
1288048,drivers/isdn/hisax/ipac.h,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,linus torvalds,1113690036,linus torvalds,1113690036,linus torvalds,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1,1,1,Unknown,2005,2005.04


In [386]:
#t.shape, mh.shape, mf.shape, len(np.unique(mf.rev))

In [387]:
mf.columns

Index([u'file', u'mtnr', u'rev', u'status', u'subsys'], dtype='object')

In [388]:
mf.columns, t.columns

(Index([u'file', u'mtnr', u'rev', u'status', u'subsys'], dtype='object'),
 Index([u'rev', u'ctm', u'year'], dtype='object'))

In [389]:
x = mf.merge(t, on=['rev'], how='left')

In [390]:
x.columns

Index([u'file', u'mtnr', u'rev', u'status', u'subsys', u'ctm', u'year'], dtype='object')

In [427]:
x.head()

,file,mtnr,rev,status,subsys,ctm,year
0,.gitignore,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST,1389063267,2014
1,.mailmap,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST,1389063267,2014
2,COPYING,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST,1389063267,2014
3,CREDITS,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST,1389063267,2014
4,Documentation/.gitignore,rlandley@parallels.com,81df63a9a5f98edbfaae70729147a33a3b7690a4,Maintained,DOCUMENTATION,1389063267,2014


In [350]:
x.isnull().sum()

file      0
mtnr      0
rev       0
status    0
subsys    0
ctm       0
year      0
dtype: int64

In [362]:
dict_my_by_maintained = dict()

In [363]:
for rev in mtnr_rev.keys():
    try:
        y = mh['year'][mh['rev'] == rev].iloc[0] # mh may not have record of rev
    except:
        continue
    mrev = mtnr_rev[rev]
    for sysname in mrev:
        subsys = mrev[sysname]
        for m in subsys.mtnrs.keys():
            my = (m, y)
            if my not in dict_my_by_maintained:
                dict_my_by_maintained[my] = {'subsyses':set(), 
                                             'file_list':set(), 'cfile_list':set(),
                                             'subsys_file_list':collections.defaultdict(set), 
                                             'subsys_supported_file_list':collections.defaultdict(set), 
                                             'subsys_cfile_list':collections.defaultdict(set), 
                                             'subsys_supported_cfile_list':collections.defaultdict(set)}

In [411]:
newi = i

In [487]:
x.head()

,file,mtnr,rev,status,subsys,ctm,year
0,.gitignore,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST,1389063267,2014
1,.mailmap,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST,1389063267,2014
2,COPYING,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST,1389063267,2014
3,CREDITS,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST,1389063267,2014
4,Documentation/.gitignore,rlandley@parallels.com,81df63a9a5f98edbfaae70729147a33a3b7690a4,Maintained,DOCUMENTATION,1389063267,2014


In [412]:
i, maxi

(216055861, 217284141)

In [374]:
len(dict_my_by_maintained.keys())

7833

In [376]:
dict_my_by_maintained[dict_my_by_maintained.keys()[0]]

{'added_cfile_list': {'drivers/staging/rtl8712/hal_init.c',
  'drivers/staging/rtl8712/ieee80211.c',
  'drivers/staging/rtl8712/mlme_linux.c',
  'drivers/staging/rtl8712/os_intfs.c',
  'drivers/staging/rtl8712/recv_linux.c',
  'drivers/staging/rtl8712/rtl8712_cmd.c',
  'drivers/staging/rtl8712/rtl8712_efuse.c',
  'drivers/staging/rtl8712/rtl8712_io.c',
  'drivers/staging/rtl8712/rtl8712_led.c',
  'drivers/staging/rtl8712/rtl8712_recv.c',
  'drivers/staging/rtl8712/rtl8712_xmit.c',
  'drivers/staging/rtl8712/rtl871x_cmd.c',
  'drivers/staging/rtl8712/rtl871x_eeprom.c',
  'drivers/staging/rtl8712/rtl871x_io.c',
  'drivers/staging/rtl8712/rtl871x_ioctl_linux.c',
  'drivers/staging/rtl8712/rtl871x_ioctl_rtl.c',
  'drivers/staging/rtl8712/rtl871x_ioctl_set.c',
  'drivers/staging/rtl8712/rtl871x_mlme.c',
  'drivers/staging/rtl8712/rtl871x_mp.c',
  'drivers/staging/rtl8712/rtl871x_mp_ioctl.c',
  'drivers/staging/rtl8712/rtl871x_pwrctrl.c',
  'drivers/staging/rtl8712/rtl871x_recv.c',
  'driver

In [410]:
newi, i, x.shape

(20473512, 216055861, (217284141, 7))

In [413]:
maxi = x.shape[0]
for i, idx in enumerate(x.index):
    if i % 2000000 == 0:
        print i / 2000000,
    if i < newi:
        continue
    m, y, ss, f, st = x.get_value(idx, 'mtnr'), x.get_value(idx, 'year'), x.get_value(idx, 'subsys'), x.get_value(idx, 'file'), x.get_value(idx, 'status')
    my = (m, y)
    if my not in dict_my_by_maintained:
        dict_my_by_maintained[my] = {'subsyses':set(), 
                                     'file_list':set(), 'cfile_list':set(),
                                     'subsys_file_list':collections.defaultdict(set), 
                                     'subsys_supported_file_list':collections.defaultdict(set), 
                                     'subsys_cfile_list':collections.defaultdict(set), 
                                     'subsys_supported_cfile_list':collections.defaultdict(set)}
    dict_my_by_maintained[my]['subsyses'].add(ss)
    dict_my_by_maintained[my]['file_list'].add(f)
    dict_my_by_maintained[my]['subsys_file_list'][ss].add(f)
    is_supported = st == 'Supported'
    if is_supported:
        dict_my_by_maintained[my]['subsys_supported_file_list'][ss].add(f)
    if f[-1] in ['c', 'C']:
        dict_my_by_maintained[my]['cfile_list'].add(f)
        dict_my_by_maintained[my]['subsys_cfile_list'][ss].add(f)
        if is_supported:
            dict_my_by_maintained[my]['subsys_supported_cfile_list'][ss].add(f)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108


In [437]:
#dict_my_by_maintained[('21cnbao@gmail.com', 2011)]

In [438]:
#dict_my_by_maintained[('21cnbao@gmail.com', 2012)]

In [434]:
for my in dict_my_by_maintained.keys():
    try:
        m_prevy = (my[0], my[1] - 1)
        if m_prevy not in dict_my_by_maintained:
            dict_my_by_maintained[my]['added_file_list'] = dict_my_by_maintained[my]['file_list']
            dict_my_by_maintained[my]['deled_file_list'] = []
            dict_my_by_maintained[my]['added_cfile_list'] = dict_my_by_maintained[my]['cfile_list']
            dict_my_by_maintained[my]['deled_cfile_list'] = []
        else:
            dict_my_by_maintained[my]['added_file_list'] = list(np.setdiff1d(list(dict_my_by_maintained[my]['file_list']), list(dict_my_by_maintained[m_prevy]['file_list'])))
            dict_my_by_maintained[my]['deled_file_list'] = list(np.setdiff1d(list(dict_my_by_maintained[m_prevy]['file_list']), list(dict_my_by_maintained[my]['file_list'])))
            dict_my_by_maintained[my]['added_cfile_list'] = list(np.setdiff1d(list(dict_my_by_maintained[my]['cfile_list']), list(dict_my_by_maintained[m_prevy]['cfile_list'])))
            dict_my_by_maintained[my]['deled_cfile_list'] = list(np.setdiff1d(list(dict_my_by_maintained[m_prevy]['cfile_list']), list(dict_my_by_maintained[my]['cfile_list'])))
    except Exception, e:
            continue

In [433]:
np.setdiff1d(list({'1', '3'}), list({'1'}))

array(['3'], 
      dtype='|S1')

In [439]:
res = []

In [440]:
for my in dict_my_by_maintained:
    v_my = dict_my_by_maintained[my]
    tmp = dict()
    try:
        tmp['mtnr'], tmp['year'] = my[0], my[1]
    except Exception:
        continue
    tmp['num_subsyses'] = len(v_my['subsyses'])
    tmp['subsys_list'] = '#'.join(v_my['subsyses'])
    subsys_f_cnt = 0
    for (modi, k) in [('mod1', 'subsys_file_list'), ('mod2', 'subsys_supported_file_list'), ('mod1c', 'subsys_cfile_list'), ('mod2c', 'subsys_supported_cfile_list')]:
        t = v_my[k]
        tmp['num_' + k[7:-5]] = len(reduce(lambda x, y : set(list(x) + list(y)), t.values(), []))
        if len(t) <= 0:
            continue
        s_cnt = Series(map(lambda v:len(t[v]), t.keys()), index=t.keys())
        tmp[modi] = sys2mod[s_cnt.idxmax()]
    
    for k in ['added_file_list', 'deled_file_list', 'added_cfile_list', 'deled_cfile_list']:
        tmp['num_' + k[:-5]] = len(v_my[k])
    
    res.append(tmp)    

In [441]:
#stat_my_by_maintained = DataFrame([{'mtnr':k[0], 'year':k[1], 'numfiles':len(v['files']), 'numcfiles':len(v['cfiles']), 'numsubsyses':len(v['subsyses'])} for k, v in dict_my_by_maintained.items()])
stat_my_by_maintained = DataFrame(res)

In [442]:
stat_my_by_maintained.shape

(7833, 16)

In [443]:
stat_my_by_maintained.sort_values(['mtnr', 'year'], inplace=True)

In [445]:
stat_my_by_maintained.head()

,mod1,mod1c,mod2,mod2c,mtnr,num_added_cfile,num_added_file,num_cfile,num_deled_cfile,num_deled_file,num_file,num_subsyses,num_supported_cfile,num_supported_file,subsys_list,year
4359,drivers,drivers,NaN,NaN,21cnbao@gmail.com,10,28,10,0,0,28,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2011
2276,drivers,drivers,NaN,NaN,21cnbao@gmail.com,4,6,14,0,1,33,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2012
2584,drivers,drivers,NaN,NaN,21cnbao@gmail.com,14,17,26,2,6,44,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2013
2888,drivers,drivers,NaN,NaN,21cnbao@gmail.com,3,6,24,5,14,36,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2014
3192,drivers,drivers,NaN,NaN,21cnbao@gmail.com,2,2,14,12,14,24,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2015


In [446]:
stat_my_by_maintained.to_csv(mtnrdir + 'mtnr-yearly-stat-by-maintain.csv', mode='w', header=True, index=False, sep=';')

In [96]:
stat_my_by_maintained = pd.read_csv(mtnrdir + 'mtnr-yearly-stat-by-maintain.csv', header=0, sep=';')

In [114]:
(stat_my_by_maintained.num_file[stat_my_by_maintained.year > 2010] == 0).sum(), stat_my_by_maintained.shape, (stat_my_by_maintained.year > 2010).sum()

(163, (7833, 16), 4534)

In [115]:
#stat_my_by_maintained[stat_my_by_maintained.mtnr == '[shc_work@mail.ru]']

In [116]:
#stat_my_by_maintained[np.logical_and(stat_my_by_maintained.mod1.isnull(), stat_my_by_maintained.year >= 2013)].tail()

In [117]:
#tat_my_by_maintained[stat_my_by_maintained.mtnr=='umalhi@cisco.com']

In [467]:
s = mtnr_rev['e46d12c65998c7ac6a8544fc356e2297228b207a']['CISCO VIC LOW LATENCY NIC DRIVER']

In [469]:
s.mtnrs

{'umalhi@cisco.com': ('upinder malhi', 'umalhi@cisco.com')}

In [470]:
s.files

['drivers/infiniband/hw/usnic']

In [471]:
fcate_of_fn['usnic_debugfs.c']

KeyError: 'usnic_debugfs.c'

In [ ]:
def get_maintainer_list_and_subsys_list(fn, fcate_of_fn, fcate_2_subsys, mntr_of_rev):
    if fn not in fcate_of_fn:
        return idmp['linus torvalds'], Series()
    fcates = fcate_of_fn[fn]
    ret = []
    subsys_list = []
    for fc in fcates:
        if fc in fcate_2_subsys:
            subsys = fcate_2_subsys[fc]
            subsys_list.extend(subsys)
            for ss in subsys:
                ret.extend(mntr_of_rev[ss].mtnrs)

    return (idmp['linus torvalds'] if len(ret) == 0 else '#'.join(np.unique(ret))), Series(subsys_list)

In [982]:
mh.sys_name.str.contains('#').sum()

0

In [983]:
Series([1, 3, 2], index=['a', 'x', 'y']).idxmax()

'x'

In [984]:
{1:[12, 23], 2:[2, 3]}.values()

[[12, 23], [2, 3]]

In [609]:
mfiles_of_mtnr = collections.defaultdict(set)

In [610]:
itr = delta_file_rev_ct.iterrows()

In [611]:
while True:
    try:
        idx, onedelta = next(itr)
        for m in imlist[idx]:
            mfiles_of_mtnr[m].add(onedelta.fn)
    except StopIteration:
        break

In [674]:
mtnr_stat = DataFrame({}, index=mfiles_of_mtnr.keys())

In [675]:
mtnr_stat['num_files'] = map(lambda x:len(mfiles_of_mtnr[x]), mtnr_stat.index)

In [676]:
mtnr_stat['num_deltas_as_athr'] = (delta_file_rev_ct['aid'].value_counts())[mtnr_stat.index].fillna(0).astype(int)

In [677]:
mtnr_stat['num_deltas_as_cmtr'] = (delta_file_rev_ct['cid'].value_counts())[mtnr_stat.index].fillna(0).astype(int)

In [665]:
grp = delta_file_rev_ct.groupby('cid')
v = grp['aid'].agg(lambda x:len(np.unique(x)))


In [678]:
mtnr_stat['num_athrs'] = v[mtnr_stat.index].fillna(0).astype(int)

In [667]:
tmh = mh[np.logical_and(mh['type'] == 'M', mh['op'] =='+')]
grp = tmh.groupby('obj')
v1 = grp['time'].agg(np.min)
tmh = mh[np.logical_and(mh['type'] == 'M', mh['op'] =='-')]
grp = tmh.groupby('obj')
v2 = grp['time'].agg(np.max)

In [679]:
mtnr_stat['first_add_time'] = v1[mtnr_stat.index].fillna(0).astype(int)
mtnr_stat['last_del_time'] = v2[mtnr_stat.index].fillna(0).astype(int)

In [680]:
mtnr_stat.to_csv(mtnrdir + mtnr_stat_fn, mode='w', header=False, index=True, sep=';')

In [681]:
mtnr_stat.head()

,num_files,num_deltas_as_athr,num_deltas_as_cmtr,num_athrs,first_add_time,last_del_time
vdavydov@parallels.com,1,243,0,0,1449944134,1450101799
rlove@google.com,17,187,74,11,1113690036,1425931807
neel patel,12,8,0,0,1328389263,1332923715
james hogan,57,449,166,13,1362254956,1420928231
anisse astier,1,35,0,0,1261553193,1436865577


## monthly maintainer stat, by maintained

```
maintainer,

natural month,

#files maintained,

module (that has the largest fraction of the files he maintains at this 
month),

#authors (all) on the files he maintained,

#commits on the files he maintained,

#commits he authored on the files he maintains,

#commits authored by other maintainers (who maintain other files),

# (all) commits he authored,

the first time he appeared on MAINTAINERS file,

the last time he was not on the MAINTAINERS file (if there is)
```

In [140]:
mf.head()

,file,mtnr,rev,status,subsys
0,.gitignore,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST
1,.mailmap,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST
2,COPYING,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST
3,CREDITS,linus torvalds,81df63a9a5f98edbfaae70729147a33a3b7690a4,Unknown,THE REST
4,Documentation/.gitignore,rlandley@parallels.com,81df63a9a5f98edbfaae70729147a33a3b7690a4,Maintained,DOCUMENTATION


In [268]:
tdf = delta[delta.fn == 'MAINTAINERS'][['rev', 'ctm', 'ym']].drop_duplicates()

x = mf.merge(tdf, on=['rev'], how='left')

dict_mym_by_maintained = dict()

In [269]:
for rev in mtnr_rev.keys():
    try:
        ym = mh['ym'][mh['rev'] == rev].iloc[0] # mh may not have record of rev
    except:
        continue
    mrev = mtnr_rev[rev]
    for sysname in mrev:
        subsys = mrev[sysname]
        for m in subsys.mtnrs.keys():
            mym = (m, ym)
            if mym not in dict_mym_by_maintained:
                dict_mym_by_maintained[mym] = {'subsyses':set(), 
                                               'subsyses_supported':set(),
                                               'file_list':set(), 'cfile_list':set(),
                                               'subsys_file_list':collections.defaultdict(set), 
                                               'subsys_supported_file_list':collections.defaultdict(set), 
                                               'subsys_cfile_list':collections.defaultdict(set), 
                                               'subsys_supported_cfile_list':collections.defaultdict(set)}

In [ ]:
maxi = x.shape[0]
newi = 0

In [ ]:
for i, idx in enumerate(x.index):
    if i % 2000000 == 0:
        print i / 2000000,
    if i < newi:
        continue
    m, ym, ss, f, st = x.get_value(idx, 'mtnr'), x.get_value(idx, 'ym'), x.get_value(idx, 'subsys'), x.get_value(idx, 'file'), x.get_value(idx, 'status')
    mym = (m, ym)
    if mym not in dict_mym_by_maintained:
        dict_mym_by_maintained[mym] = {'subsyses':set(),
                                       'subsyses_supported':set(),
                                       'file_list':set(), 'cfile_list':set(),
                                       'subsys_file_list':collections.defaultdict(set), 
                                       'subsys_supported_file_list':collections.defaultdict(set), 
                                       'subsys_cfile_list':collections.defaultdict(set), 
                                       'subsys_supported_cfile_list':collections.defaultdict(set)}
    dict_mym_by_maintained[mym]['subsyses'].add(ss)
    dict_mym_by_maintained[mym]['file_list'].add(f)
    dict_mym_by_maintained[mym]['subsys_file_list'][ss].add(f)
    is_supported = st == 'Supported'
    if is_supported:
        dict_mym_by_maintained[mym]['subsyses_supported'].add(ss)
        dict_mym_by_maintained[mym]['subsys_supported_file_list'][ss].add(f)
        
    if f[-1] in ['c', 'C']:
        dict_mym_by_maintained[mym]['cfile_list'].add(f)
        dict_mym_by_maintained[mym]['subsys_cfile_list'][ss].add(f)
        if is_supported:
            dict_mym_by_maintained[mym]['subsys_supported_cfile_list'][ss].add(f)

0

In [ ]:
def get_prev_month(mth):
    if 0.0098 <= mth - int(mth) <= 0.012:
        return mth - 1 + 0.11
    return mth -  0.01

def get_next_month(mth):
    nmth = mth + 0.01
    if nmth - int(nmth) >= 0.129: # 0.13
        return int(mth) + 1.01
    return nmth

In [ ]:
for my in dict_mym_by_maintained.keys():
    try:
        m_prevy = (my[0], get_prev_month(my[1]))
        if m_prevy not in dict_mym_by_maintained:
            dict_mym_by_maintained[my]['added_file_list'] = dict_mym_by_maintained[my]['file_list']
            dict_mym_by_maintained[my]['deled_file_list'] = []
            dict_mym_by_maintained[my]['added_cfile_list'] = dict_mym_by_maintained[my]['cfile_list']
            dict_mym_by_maintained[my]['deled_cfile_list'] = []
        else:
            dict_mym_by_maintained[my]['added_file_list'] = list(np.setdiff1d(list(dict_mym_by_maintained[my]['file_list']), list(dict_mym_by_maintained[m_prevy]['file_list'])))
            dict_mym_by_maintained[my]['deled_file_list'] = list(np.setdiff1d(list(dict_mym_by_maintained[m_prevy]['file_list']), list(dict_mym_by_maintained[my]['file_list'])))
            dict_mym_by_maintained[my]['added_cfile_list'] = list(np.setdiff1d(list(dict_mym_by_maintained[my]['cfile_list']), list(dict_mym_by_maintained[m_prevy]['cfile_list'])))
            dict_mym_by_maintained[my]['deled_cfile_list'] = list(np.setdiff1d(list(dict_mym_by_maintained[m_prevy]['cfile_list']), list(dict_mym_by_maintained[my]['cfile_list'])))
    except Exception, e:
            continue

In [86]:
len(dict_mym_by_maintained)

82971

In [87]:
mtnr_months = sorted({mym[1] for mym in dict_mym_by_maintained})
min_month = min(mtnr_months)
max_month = max(mtnr_months)

mtnrs_set = set([mym[0] for mym in dict_mym_by_maintained])



# for m in mtnrs_set:
#     prev_month = min_month
#     mymp = (m, prev_month)
#     now_month = get_next_month(prev_month)
#     while now_month <= max_motnh:
#         mym = (m, now_month)
#         if mymp in dict_mym_by_maintained and mym not in dict_mym_by_maintained:
#             dict_mym_by_maintained[mym] = copy.deepcopy(dict_mym_by_maintained[mymp])
#         prev_month, mymp = now_month, mym
#         now_month = get_next_month(prev_month)

prev_month = min_month
now_month = get_next_month(prev_month)
while now_month <= max_month:
    if now_month not in mtnr_months:
        for m in mtnrs_set:
            mymp = (m, prev_month)
            if mymp in dict_mym_by_maintained:
                mym = (m, now_month)
                dict_mym_by_maintained[mym] = copy.deepcopy(dict_mym_by_maintained[mymp])
    prev_month = now_month
    now_month = get_next_month(prev_month)

In [88]:
len(dict_mym_by_maintained)

82971

In [ ]:
for mym in dict_mym_by_maintained:
    for key in ['authors_f', 'commits_f', 'commits_authored_f', 'commits_authored_by_other_mtnrs_f', 'commits_authored']:
        dict_mym_by_maintained[mym][key] = set()

In [ ]:
newi = 0

In [120]:
delta.shape

(1305334, 14)

In [ ]:
mlist_of_month = collections.defaultdict(list)
for mym in dict_mym_by_maintained:
    mlist_of_month[mym[1]].append(mym[0])

In [ ]:
for i, idx in enumerate(delta.index):
    if i % 10000 == 0:
        print i / 10000,
    if i < newi:
        continue
    fn, rev, aid, ym = delta.get_value(idx, 'fn'), delta.get_value(idx, 'rev'), delta.get_value(idx, 'aid'), delta.get_value(idx, 'ym')
    mlist = imlist[idx]
    aym = (aid, ym)
    if aym in dict_mym_by_maintained:
        dict_mym_by_maintained[aym]['commits_authored'].add(rev)
    for m in mlist:
        mym = (m, ym)
        if mym in dict_mym_by_maintained: # it can be not true
            dict_mym_by_maintained[mym]['authors_f'].add(aid)
            dict_mym_by_maintained[mym]['commits_f'].add(rev)
            if m == aid:
                dict_mym_by_maintained[mym]['commits_authored_f'].add(rev)
            elif aid in mlist_of_month[ym]:
                dict_mym_by_maintained[mym]['commits_authored_by_other_mtnrs_f'].add(rev)    
             
    

In [162]:
# for mth in mtnr_months:
#     print mth, 'maciej sosnowski' in mlist_of_month[mth]

In [131]:
i, idx

(498670, 806661)

In [158]:
time.gmtime(1261344247), time.gmtime(1267457742)

(time.struct_time(tm_year=2009, tm_mon=12, tm_mday=20, tm_hour=21, tm_min=24, tm_sec=7, tm_wday=6, tm_yday=354, tm_isdst=0),
 time.struct_time(tm_year=2010, tm_mon=3, tm_mday=1, tm_hour=15, tm_min=35, tm_sec=42, tm_wday=0, tm_yday=60, tm_isdst=0))

In [161]:
time.gmtime(1267126770), time.gmtime(1267414062)

(time.struct_time(tm_year=2010, tm_mon=2, tm_mday=25, tm_hour=19, tm_min=39, tm_sec=30, tm_wday=3, tm_yday=56, tm_isdst=0),
 time.struct_time(tm_year=2010, tm_mon=3, tm_mday=1, tm_hour=3, tm_min=27, tm_sec=42, tm_wday=0, tm_yday=60, tm_isdst=0))

In [134]:
get_maintainer_list_and_subsys_list('drivers/dma/fsldma.c', fcate_of_fn, fcate_2_subsys, mntr_of_rev=mtnr_rev['a040d532b912b5dd7d88692b580cff9c88b987e3'])

('dan williams#leo@zh-kernel.org#maciej sosnowski#wei.zhang@freescale.com',
 0    ASYNCHRONOUS TRANSFERS/TRANSFORMS (IOAT) API
 1            DMA GENERIC OFFLOAD ENGINE SUBSYSTEM
 2                            FREESCALE DMA DRIVER
 dtype: object)

In [ ]:
# get module
for mym in dict_mym_by_maintained:
    dict_mym_by_maintained[mym]['mod'] = calc_sys2mod(sysname=None, flist=dict_mym_by_maintained[mym]['file_list'])

In [ ]:
res = []

In [ ]:
for mym in dict_mym_by_maintained:
    d = dict_mym_by_maintained[mym]
    tmp = {'mtnr':mym[0],
           'month':mym[1],
           'mod':d['mod'],
           'num_files_maintained': len(d['file_list']), 
           'num_cfiles_maintained': len(d['cfile_list']),
           'num_authors_on_maintained_file': len(d['authors_f']), 
           'num_commits_on_maintained_file': len(d['commits_f']), 
           'num_commits_he_authored_on_maintained__files': len(d['commits_authored_f']), 
           'num_commits_authored_by_other_maintainers': len(d['commits_authored_by_other_mtnrs_f']), 
           'num_commits_he_authored': len(d['commits_authored'])}
    
    v_my = dict_mym_by_maintained[mym]
    try:
        tmp['mtnr'], tmp['ym'] = mym[0], mym[1]
    except Exception:
        continue
    tmp['num_subsyses'] = len(v_my['subsyses'])
    tmp['num_subsyses_supported'] = len(v_my['subsyses_supported'])
    tmp['subsys_list'] = '#'.join(v_my['subsyses'])
    tmp['subsys_list_supported'] = '#'.join(v_my['subsyses_supported'])
    subsys_f_cnt = 0
    for (modi, k) in [('mod1', 'subsys_file_list'), ('mod2', 'subsys_supported_file_list'), ('mod1c', 'subsys_cfile_list'), ('mod2c', 'subsys_supported_cfile_list')]:
        t = v_my[k]
        tmp['num_' + k[7:-5]] = len(reduce(lambda x, y : set(list(x) + list(y)), t.values(), []))
        if len(t) <= 0:
            continue
        s_cnt = Series(map(lambda v:len(t[v]), t.keys()), index=t.keys())
        tmp[modi] = sys2mod[s_cnt.idxmax()]
    
    for k in ['added_file_list', 'deled_file_list', 'added_cfile_list', 'deled_cfile_list']:
        tmp['num_' + k[:-5]] = len(v_my[k])
        
    res.append(tmp)

In [ ]:
mtnr_monthly_stat_by_maintain = DataFrame(res)

In [259]:
# mtnr_monthly_stat_by_maintain.shape, np.nansum(mtnr_monthly_stat_by_maintain['mod'] == mtnr_monthly_stat_by_maintain['mod1'])

In [257]:
np.nan!= np.nan

True

#### the first time he appeared on MAINTAINERS file,

#### the last time he was not on the MAINTAINERS file (if there is)


In [182]:
mh.head()

,sys_name,type,obj,op,rev,time,m_name,m_email,year,ym
0,3C359 NETWORK DRIVER,S,3C359 NETWORK DRIVER,+,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1.113690e+09,,,2005,2005.04
1,3C359 NETWORK DRIVER,M,mike phillips,+,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1.113690e+09,mike phillips,,2005,2005.04
2,3C359 NETWORK DRIVER,ST,Maintained,+,1da177e4c3f41524e886b7f1b8a0c1fc7321cac2,1.113690e+09,,,2005,2005.04
3,3C359 NETWORK DRIVER,ST,<__main__.SubSys instance at 0x7fa5df3907e8>,+,9e95ce279fa611226a1ab0dff1c237c080b51b60,1.113690e+09,,,2005,2005.04
4,3C359 NETWORK DRIVER,ST,<__main__.SubSys instance at 0x7fa559bdf248>,-,9e95ce279fa611226a1ab0dff1c237c080b51b60,1.113690e+09,,,2005,2005.04


In [ ]:
tmh = mh[np.logical_and(mh['type'] == 'M', mh['op'] == '+')]
grp = tmh.groupby('obj')
time_first_add = grp['time'].agg(np.min)
time_last_add = grp['time'].agg(np.max)

tmh = mh[np.logical_and(mh['type'] == 'M', mh['op'] == '-')]
grp = tmh.groupby('obj')
time_last_del = grp['time'].agg(np.max)


In [261]:
time_last_add['3chas3@gmail.com']

1113690036.0

In [ ]:
for col in ['time_first_add', 'time_last_add', 'time_last_del']:
    mtnr_monthly_stat_by_maintain[col] = eval(col + '[mtnr_monthly_stat_by_maintain["mtnr"]].values')
    #mtnr_monthly_stat_by_maintain[col] = mtnr_monthly_stat_by_maintain[col].astype(int)

In [263]:
(mtnr_monthly_stat_by_maintain.time_last_add > mtnr_monthly_stat_by_maintain.time_last_del).sum()

37653

In [ ]:
mtnr_monthly_stat_by_maintain.sort_values(['mtnr', 'month'], inplace=True)

In [267]:
mtnr_monthly_stat_by_maintain.head()

,mod,mod1,mod1c,mod2,mod2c,month,mtnr,num_added_cfile,num_added_file,num_authors_on_maintained_file,...,num_file,num_files_maintained,num_subsyses,num_supported_cfile,num_supported_file,subsys_list,ym,time_first_add,time_last_add,time_last_del
32150,arch,drivers,drivers,NaN,NaN,2011.07,21cnbao@gmail.com,7,23,0,...,23,23,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2011.07,1.310487e+09,1.312520e+09,1.312413e+09
35689,arch,drivers,drivers,NaN,NaN,2011.08,21cnbao@gmail.com,0,0,1,...,23,23,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2011.08,1.310487e+09,1.312520e+09,1.312413e+09
65302,arch,drivers,drivers,NaN,NaN,2011.09,21cnbao@gmail.com,0,0,5,...,23,23,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2011.09,1.310487e+09,1.312520e+09,1.312413e+09
78395,arch,drivers,drivers,NaN,NaN,2011.10,21cnbao@gmail.com,0,0,3,...,23,23,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2011.10,1.310487e+09,1.312520e+09,1.312413e+09
40331,arch,drivers,drivers,NaN,NaN,2011.11,21cnbao@gmail.com,3,5,2,...,28,28,1,0,0,ARM/CSR SIRFPRIMA2 MACHINE SUPPORT,2011.11,1.310487e+09,1.312520e+09,1.312413e+09


In [ ]:
mtnr_monthly_stat_by_maintain['month'] = map(lambda v : '{:0<7}'.format(v), mtnr_monthly_stat_by_maintain['month'])

In [ ]:
mtnr_monthly_stat_by_maintain.to_csv(mtnrdir + 'mtnr_monthly_stat_by_maintain_more.csv', header=True, index=False, sep=';')

In [287]:
1

1